<a href="https://colab.research.google.com/github/Gretarobur/pyqb-20220714/blob/main/Quantidiche_IF_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math
import matplotlib.pyplot as plt
import os
from collections import Counter
!pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.4 MB/s eta 0:00:00


In [ ]:
#@title !!! Voxel Size: Inserire la dimensione dei pixel in X/Y e Z (Info dell'immagine) !!!
XY_pixel = 0.0964 #@param {type:"raw"}
Z_pixel = 0.4196 #@param {type:"raw"}

voxel_size = XY_pixel * XY_pixel * Z_pixel

In [ ]:
#@title !!! Scrivere il canale che si vuole quantificare (KAP1/HP1/BRD4) !!!
Canale = "SUZ12" #@param {type:"string"}
if Canale == "KAP1":
  Canale = 1
elif Canale == "HP1" or "BRD4" or "SUZ12":
  Canale = 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
directory = "/content/drive/MyDrive/Analysis"
files = os.listdir(directory)

fields = []
for subfolder in files:
  fields.append(subfolder)

directory_analysis = []
for subfolder in files:
  directory_analysis.append(directory + "/" + subfolder)

In [ ]:
alist = os.listdir(directory_analysis[0])

alist.sort()
alist

['M_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos005_S001.tif.csv',
 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos005_S001.tif KAP1.csv',
 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos005_S001.tif SUZ12.csv',
 'serie_05.csv']

In [ ]:
field = []
for subfolder in directory_analysis:
  field.append(subfolder)

In [ ]:
def la_distanza(il_pallino, il_nucleo):
  return math.sqrt((il_nucleo[0] - il_pallino[0])**2 + (il_nucleo[1] - il_pallino[1])**2)

def distance_3D(il_pallino, nuclei_data):
  return math.sqrt((nuclei_data[1] - il_pallino[0])**2 + (nuclei_data[2] - il_pallino[1])**2 + (nuclei_data[3] - il_pallino[2])**2)

In [ ]:
list_merged = []
dict_merged = {}
for experiment in directory_analysis:

  # Lista dei centroidi dei nuclei
  # Tutte le distanze/volumi sono in pixel/voxel, sono da moltiplicare per la dimensione reale
  # Nuclei = [(X_centroid, Y_centroid, Z_centroid, X_max, X_min, Y_max, Y_min, Z_max, Z_min, volume)]
  #nuclei = [("N_nuclei", "x_c_nuclei", "y_c_nuclei", "z_c_nuclei", "volume_nuclei", "X_max", "X_min", "Y_max", "Y_min", "Z_max", "Z_min")]


  csv = os.listdir(experiment)
  file_path = experiment + "/" + csv[1]
  csv.sort()
  print(experiment)
  xlsx_file_path = experiment + "/" + experiment.split("/")[-1] + ".xlsx"
  print(xlsx_file_path)
  xlsx_file = pd.ExcelWriter(xlsx_file_path, engine='xlsxwriter')

  print(csv)

  nome_file_nuclei = experiment + "/" + csv[0]
  nome_file_droplet = experiment + "/" + csv[3]
  nome_file_quantifiche_nucleo_canale = experiment + "/" + csv[Canale]

  nuclei_data = []

  N_nuclei = pd.read_csv(nome_file_nuclei)['Nb'].tolist()
  x_c_nuclei = pd.read_csv(nome_file_nuclei)['CX (unit)'].tolist()
  y_c_nuclei = pd.read_csv(nome_file_nuclei)['CY (unit)'].tolist()
  z_c_nuclei = pd.read_csv(nome_file_nuclei)['CZ (unit)'].tolist()
  volume_nuclei = pd.read_csv(nome_file_nuclei)['Vol (unit)'].tolist()

  N_nuclei_canale = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Nb'].tolist()
  MFI = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Mean'].tolist()

  channel_data = [(N_nuclei_canale[i], MFI[i]) for i in range(0, len(x_c_nuclei))]

  #X_max = pd.read_csv(nome_file_nuclei)['Xmax (pix)'].tolist()
  #X_min = pd.read_csv(nome_file_nuclei)['Xmin (pix)'].tolist()
  #Y_max = pd.read_csv(nome_file_nuclei)['Ymax (pix)'].tolist()
  #Y_min = pd.read_csv(nome_file_nuclei)['Ymin (pix)'].tolist()
  #Z_max = pd.read_csv(nome_file_nuclei)['Zmax (pix)'].tolist()
  #Z_min = pd.read_csv(nome_file_nuclei)['Zmin (pix)'].tolist()

  #nuclei_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i], volume_nuclei[i], X_max[i], X_min[i], Y_max[i], Y_min[i], Z_max[i], Z_min[i]) for i in range(0, len(x_c_nuclei))]
  nuclei_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i], volume_nuclei[i]) for i in range(0, len(x_c_nuclei))]

  # Droplet = [(x_c_droplet, y_c_droplet, z_c_droplet, droplet_fluorescence droplet_volume, nucleo associato, x del nucleo, y del nucleo, distanza dal centroide)]

  x_c_droplet_pix = pd.read_csv(nome_file_droplet)['X'].tolist()
  x_c_droplet = [i * XY_pixel for i in x_c_droplet_pix]
  y_c_droplet_pix = pd.read_csv(nome_file_droplet)['Y'].tolist()
  y_c_droplet = [i * XY_pixel for i in y_c_droplet_pix]
  z_c_droplet_pix = pd.read_csv(nome_file_droplet)['Z'].tolist()
  z_c_droplet = [i * Z_pixel for i in z_c_droplet_pix]
  droplet_volume_vox = pd.read_csv(nome_file_droplet)['Size'].tolist()
  droplet_volume = [i * voxel_size for i in droplet_volume_vox]
  droplet_fluorescence = pd.read_csv(nome_file_droplet)['Av'].tolist()

  #le_aree = pd.read_csv(nome_file_droplet + '.csv')['Size'].tolist()
  #le_fluorescenze = pd.read_csv(nome_file_droplet + '.csv')['Av'].tolist()
  droplets = [[x_c_droplet[i], y_c_droplet[i], z_c_droplet[i], droplet_volume[i], droplet_fluorescence[i], 0, 0, 0, 0, 0, None, None, 0] for i in range(0, len(x_c_droplet))]
  droplets_list = [[x_c_droplet[i], y_c_droplet[i], z_c_droplet[i], droplet_volume[i], droplet_fluorescence[i], 0, 0, 0, 0, 0, None, None, 0] for i in range(0, len(x_c_droplet))]

  Nuclei_DF = pd.DataFrame (nuclei_data, columns = ["Nb", "Cx", "Cy","Cz","Volume_unit"])
  Nuclei_DF["Nb"] = Nuclei_DF["Nb"] + 1
  Nuclei_DF['Channel intensisty'] = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Mean'].tolist()
  Nuclei_DF.drop(Nuclei_DF[Nuclei_DF['Volume_unit'] <= 10].index, inplace = True)
  Nuclei_DF["Volume_pix"] = Nuclei_DF["Volume_unit"] / voxel_size

  if len(Nuclei_DF) == 0:
    with open(experiment + '/log.txt', 'w') as f:
      f.write('No nuclei')
  else:
    Droplets_DF = pd.DataFrame (droplets_list, columns = ["Cx","Cy","Cz","Volume_unit","Fluorescence","Centroid_distance","Nucleus_ID","0","0","0","Nucleo","N_Dots","0"])

    conta_dxn = {}
    nuclei_index = []
    min_distance = []
    Nuclei_index_touse = Nuclei_DF.Nb.tolist()
    for i, row_D in Droplets_DF.iterrows():
      distances = []
      for k, row_N in Nuclei_DF.iterrows():
        droplet_distance = distance_3D (row_D,row_N)
        distances.append(droplet_distance)
      index_min = distances.index(min(distances))
      min_distance.append(min(distances))
      nuclei_index.append(Nuclei_index_touse[index_min])

    Droplets_DF["Centroid_distance"] = min_distance
    Droplets_DF["Nucleus_ID"] = nuclei_index
    Droplets_DF["Nucleus_ID"] = Droplets_DF["Nucleus_ID"]
    Droplets_DF.drop(Droplets_DF[Droplets_DF['Centroid_distance'] >= 20].index, inplace = True)

    conta_dxn = Counter(Droplets_DF["Nucleus_ID"])
    keys = list(conta_dxn.keys())
    keys.sort()
    sorted_conta_dxn = {i: conta_dxn[i] for i in keys}

    key_list = list(sorted_conta_dxn.keys())
    value_list = list(sorted_conta_dxn.values())
    Dots_Nucleus = pd.DataFrame(key_list, columns = ["Nucleo"])
    Dots_Nucleus["Dots"] = value_list

    set_nuclei = set(Droplets_DF["Nucleus_ID"])
    Droplets_DF = pd.concat([Droplets_DF, Dots_Nucleus], axis=1)

    for n_nucleo in set_nuclei:
      Droplets_DF.loc[Droplets_DF.Nucleus_ID == n_nucleo,"partition ratio"] = Droplets_DF["Fluorescence"] / float(Nuclei_DF[Nuclei_DF['Nb'] == n_nucleo]['Channel intensisty'])

    Droplets_DF.to_excel(xlsx_file, sheet_name = "analysis")

    xlsx_file.save()



/content/drive/MyDrive/Analysis/Serie5
/content/drive/MyDrive/Analysis/Serie5/Serie5.xlsx
['M_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos005_S001.tif.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos005_S001.tif KAP1.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos005_S001.tif SUZ12.csv', 'serie_05.csv']


<ipython-input-9-d98922fb08af>:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  xlsx_file.save()


/content/drive/MyDrive/Analysis/Serie3
/content/drive/MyDrive/Analysis/Serie3/Serie3.xlsx
['M_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos003_S001.tif.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos003_S001.tif KAP1.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos003_S001.tif SUZ12.csv', 'serie_03.csv']


<ipython-input-9-d98922fb08af>:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  xlsx_file.save()


/content/drive/MyDrive/Analysis/Serie6_ex
/content/drive/MyDrive/Analysis/Serie6_ex/Serie6_ex.xlsx
['M_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos006_S001 tex.tif.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos006_S001 tex.tif KAP1.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos006_S001 tex.tif SUZ12.csv', 'serie_06.csv']


...

<ipython-input-9-d98922fb08af>:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  xlsx_file.save()


/content/drive/MyDrive/Analysis/Serie8
/content/drive/MyDrive/Analysis/Serie8/Serie8.xlsx
['M_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos008_S001.tif.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos008_S001.tif KAP1.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos008_S001.tif SUZ12.csv', 'serie_08.csv']


<ipython-input-9-d98922fb08af>:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  xlsx_file.save()


/content/drive/MyDrive/Analysis/Serie7
/content/drive/MyDrive/Analysis/Serie7/Serie7.xlsx
['M_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos007_S001.tif.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos007_S001.tif KAP1.csv', 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos007_S001.tif SUZ12.csv', 'serie_07.csv']


<ipython-input-9-d98922fb08af>:113: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  xlsx_file.save()


In [ ]:
csv

['M_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos007_S001.tif.csv',
 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos007_S001.tif KAP1.csv',
 'Q_5122_eff_tex_kap_suz12 - Mark_and_Find_001-Pos007_S001.tif SUZ12.csv',
 'serie_07.csv']

In [ ]:
"""
for experiment in directory_analysis:

  # Lista dei centroidi dei nuclei
  # Tutte le distanze/volumi sono in pixel/voxel, sono da moltiplicare per la dimensione reale
  # Nuclei = [(X_centroid, Y_centroid, Z_centroid, X_max, X_min, Y_max, Y_min, Z_max, Z_min, volume)]
  #nuclei = [("N_nuclei", "x_c_nuclei", "y_c_nuclei", "z_c_nuclei", "volume_nuclei", "X_max", "X_min", "Y_max", "Y_min", "Z_max", "Z_min")]

  csv = os.listdir(experiment)
  file_path = experiment + "/" + csv[1]
  csv.sort()
  print(experiment)
  xlsx_file_path = experiment + "/" + experiment.split("/")[-1] + ".xlsx"
  print(xlsx_file_path)
  xlsx_file = pd.ExcelWriter(xlsx_file_path, engine='xlsxwriter')

  print(csv)

  nome_file_nuclei = experiment + "/" + csv[1]
  nome_file_droplet = experiment + "/" + csv[0]
  nome_file_quantifiche_nucleo_canale = experiment + "/" + csv[3]

  nuclei_data = []

  N_nuclei = pd.read_csv(nome_file_nuclei)['Nb'].tolist()
  x_c_nuclei = pd.read_csv(nome_file_nuclei)['CX (unit)'].tolist()
  y_c_nuclei = pd.read_csv(nome_file_nuclei)['CY (unit)'].tolist()
  z_c_nuclei = pd.read_csv(nome_file_nuclei)['CZ (unit)'].tolist()
  volume_nuclei = pd.read_csv(nome_file_nuclei)['Vol (unit)'].tolist()

  N_nuclei_canale = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Nb'].tolist()
  MFI = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Mean'].tolist()

  channel_data = [(N_nuclei_canale[i], MFI[i]) for i in range(0, len(x_c_nuclei))]

  #X_max = pd.read_csv(nome_file_nuclei)['Xmax (pix)'].tolist()
  #X_min = pd.read_csv(nome_file_nuclei)['Xmin (pix)'].tolist()
  #Y_max = pd.read_csv(nome_file_nuclei)['Ymax (pix)'].tolist()
  #Y_min = pd.read_csv(nome_file_nuclei)['Ymin (pix)'].tolist()
  #Z_max = pd.read_csv(nome_file_nuclei)['Zmax (pix)'].tolist()
  #Z_min = pd.read_csv(nome_file_nuclei)['Zmin (pix)'].tolist()

  #nuclei_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i], volume_nuclei[i], X_max[i], X_min[i], Y_max[i], Y_min[i], Z_max[i], Z_min[i]) for i in range(0, len(x_c_nuclei))]
  nuclei_data = [(N_nuclei[i], x_c_nuclei[i], y_c_nuclei[i], z_c_nuclei[i], volume_nuclei[i]) for i in range(0, len(x_c_nuclei))]

  # Droplet = [(x_c_droplet, y_c_droplet, z_c_droplet, droplet_fluorescence droplet_volume, nucleo associato, x del nucleo, y del nucleo, distanza dal centroide)]

  x_c_droplet_pix = pd.read_csv(nome_file_droplet)['X'].tolist()
  x_c_droplet = [i * x_pixel for i in x_c_droplet_pix]
  y_c_droplet_pix = pd.read_csv(nome_file_droplet)['Y'].tolist()
  y_c_droplet = [i * x_pixel for i in y_c_droplet_pix]
  z_c_droplet_pix = pd.read_csv(nome_file_droplet)['Z'].tolist()
  z_c_droplet = [i * z_pixel for i in z_c_droplet_pix]
  droplet_volume_vox = pd.read_csv(nome_file_droplet)['Size'].tolist()
  droplet_volume = [i * voxel_size for i in droplet_volume_vox]
  droplet_fluorescence = pd.read_csv(nome_file_droplet)['Av'].tolist()

  #le_aree = pd.read_csv(nome_file_droplet + '.csv')['Size'].tolist()
  #le_fluorescenze = pd.read_csv(nome_file_droplet + '.csv')['Av'].tolist()
  droplets = [[x_c_droplet[i], y_c_droplet[i], z_c_droplet[i], droplet_volume[i], droplet_fluorescence[i], 0, 0, 0, 0, 0, None, None, 0] for i in range(0, len(x_c_droplet))]
  droplets_list = [[x_c_droplet[i], y_c_droplet[i], z_c_droplet[i], droplet_volume[i], droplet_fluorescence[i], 0, 0, 0, 0, 0, None, None, 0] for i in range(0, len(x_c_droplet))]

  Nuclei_DF = pd.DataFrame (nuclei_data, columns = ["Nb", "Cx", "Cy","Cz","Volume_unit"])
  Nuclei_DF["Nb"] = Nuclei_DF["Nb"] + 1
  Nuclei_DF['Channel intensisty'] = pd.read_csv(nome_file_quantifiche_nucleo_canale)['Mean'].tolist()
  Nuclei_DF.drop(Nuclei_DF[Nuclei_DF['Volume_unit'] <= 10].index, inplace = True)
  Nuclei_DF["Volume_pix"] = Nuclei_DF["Volume_unit"] / voxel_size

  Droplets_DF = pd.DataFrame (droplets_list, columns = ["Cx","Cy","Cz","Volume_unit","Fluorescence","Centroid_distance","Nucleus_ID","0","0","0","Nucleo","N_Dots","0"])

  conta_dxn = {}
  nuclei_index = []
  min_distance = []
  Nuclei_index_touse = Nuclei_DF.Nb.tolist()
  for i, row_D in Droplets_DF.iterrows():
    distances = []
    for k, row_N in Nuclei_DF.iterrows():
      droplet_distance = distance_3D (row_D,row_N)
      distances.append(droplet_distance)
    index_min = distances.index(min(distances))
    min_distance.append(min(distances))
    nuclei_index.append(Nuclei_index_touse[index_min])

  Droplets_DF["Centroid_distance"] = min_distance
  Droplets_DF["Nucleus_ID"] = nuclei_index
  Droplets_DF["Nucleus_ID"] = Droplets_DF["Nucleus_ID"]
  Droplets_DF.drop(Droplets_DF[Droplets_DF['Centroid_distance'] >= 20].index, inplace = True)

  conta_dxn = Counter(Droplets_DF["Nucleus_ID"])
  keys = list(conta_dxn.keys())
  keys.sort()
  sorted_conta_dxn = {i: conta_dxn[i] for i in keys}

  key_list = list(sorted_conta_dxn.keys())
  value_list = list(sorted_conta_dxn.values())
  Dots_Nucleus = pd.DataFrame(key_list, columns = ["Nucleo"])
  Dots_Nucleus["Dots"] = value_list

  set_nuclei = set(Droplets_DF["Nucleus_ID"])
  Droplets_DF = pd.concat([Droplets_DF, Dots_Nucleus], axis=1)

  for n_nucleo in set_nuclei:

    Droplets_DF["Volume_pix"] = Droplets_DF["Volume_unit"] / voxel_size
    sum_droplet_volume = Droplets_DF[Droplets_DF["Nucleus_ID"] == n_nucleo]["Volume_pix"].sum()
    Droplets_DF['Total_intensity'] = Droplets_DF["Volume_pix"] * Droplets_DF["Fluorescence"]
    sum_droplet_fluo = Droplets_DF[Droplets_DF["Nucleus_ID"] == n_nucleo]['Total_intensity'].sum()
    Nuclei_DF['Total intensity'] = Nuclei_DF['Volume_pix'] * Nuclei_DF['Channel intensisty']
    background_int_total = Nuclei_DF[Nuclei_DF['Nb'] == n_nucleo]['Total intensity'] - sum_droplet_fluo
    Nuclei_DF.loc[Nuclei_DF.Nb == n_nucleo, "Background_intensity_total"] = background_int_total
    background_volume_pix = Nuclei_DF[Nuclei_DF["Nb"] == n_nucleo]["Volume_pix"] - sum_droplet_volume
    Nuclei_DF.loc[Nuclei_DF.Nb == n_nucleo, "Background_volume_pix"] = background_volume_pix
    #Nuclei_DF.drop(Nuclei_DF[Nuclei_DF['Background_intensity_total'] <= 0].index, inplace = True)
    Nuclei_DF["Background_MFI"] = Nuclei_DF["Background_intensity_total"] / Nuclei_DF ["Background_volume_pix"]
    MFI_bcg = Nuclei_DF[Nuclei_DF['Nb'] == n_nucleo]['Background_MFI']

    Droplets_DF.loc[Droplets_DF.Nucleus_ID == n_nucleo,"partition ratio"] = Droplets_DF["Fluorescence"] / float(Nuclei_DF[Nuclei_DF['Nb'] == n_nucleo]['Background_MFI'])

  Droplets_DF.to_excel(xlsx_file, sheet_name = "analysis")
  xlsx_file.save()
  """


'\nfor experiment in directory_analysis:\n\n  # Lista dei centroidi dei nuclei\n  # Tutte le distanze/volumi sono in pixel/voxel, sono da moltiplicare per la dimensione reale\n  # Nuclei = [(X_centroid, Y_centroid, Z_centroid, X_max, X_min, Y_max, Y_min, Z_max, Z_min, volume)]\n  #nuclei = [("N_nuclei", "x_c_nuclei", "y_c_nuclei", "z_c_nuclei", "volume_nuclei", "X_max", "X_min", "Y_max", "Y_min", "Z_max", "Z_min")]\n\n  csv = os.listdir(experiment)\n  file_path = experiment + "/" + csv[1]\n  csv.sort()\n  print(experiment)\n  xlsx_file_path = experiment + "/" + experiment.split("/")[-1] + ".xlsx"\n  print(xlsx_file_path)\n  xlsx_file = pd.ExcelWriter(xlsx_file_path, engine=\'xlsxwriter\')\n\n  print(csv)\n\n  nome_file_nuclei = experiment + "/" + csv[1]\n  nome_file_droplet = experiment + "/" + csv[0]\n  nome_file_quantifiche_nucleo_canale = experiment + "/" + csv[3]\n\n  nuclei_data = []\n\n  N_nuclei = pd.read_csv(nome_file_nuclei)[\'Nb\'].tolist()\n  x_c_nuclei = pd.read_csv(nome_fi